In [ ]:
import os

import numpy as np
import scqubits as scq
import plotly.graph_objects as go

In [ ]:
qub_name = "Design1"

os.makedirs(f"../../result/{qub_name}/image", exist_ok=True)
os.makedirs(f"../../result/{qub_name}/web", exist_ok=True)

# Scan params

In [ ]:
EJb = (3.0, 7.0)
EC = 0.8
ELb = (0.35, 0.5)
avoid_freqs = [7.2, 5.9]
# avoid_freqs = []

In [ ]:
Temp = 60e-3
Q_cap = 1.0e5
Q_ind = 1.0e7
x_qp = 1.0e-3

t1s = []
e01s = []
paramss = []

scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(EJb[0], EC, ELb[0], flux=0.5, cutoff=40, truncated_dim=15)
for EJ in np.arange(EJb[0], EJb[1], 0.1):
    for EL in np.arange(ELb[0], ELb[1], 0.01):
        fluxonium.EJ = EJ
        fluxonium.EC = EC
        fluxonium.EL = EL

        spectData = fluxonium.eigensys(evals_count=15, return_spectrumdata=True)
        eval_eng, eval_sys = spectData.energy_table, spectData.state_table
        transitions0 = eval_eng[1:] - eval_eng[0]
        transitions1 = eval_eng[2:] - eval_eng[1]

        if len(avoid_freqs) > 0:
            all_collision = True
            for freqs in avoid_freqs:
                if np.any(np.abs(transitions0 - freqs) < 0.3):
                    break
                if np.any(np.abs(transitions1 - freqs) < 0.3):
                    break
            else:
                all_collision = False
        else:
            all_collision = False

        if all_collision:
            continue

        elements = fluxonium.n_operator(energy_esys=(eval_eng, eval_sys))
        t1_eff = fluxonium.t1_effective(
            esys=(eval_eng, eval_sys),
            noise_channels=[
                ("t1_capacitive", dict(Q_cap=Q_cap)),
                ("t1_inductive", dict(Q_ind=Q_ind)),
                ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
            ],
            common_noise_options=dict(i=1, j=0, T=Temp),
        )

        e01 = elements[0, 1]

        t1s.append(t1_eff)
        e01s.append(e01)
        paramss.append((EJ, EC, EL))

In [ ]:
import pandas as pd

import plotly.express as px

# 將數據轉換為DataFrame
data = pd.DataFrame(
    {
        "Matrix Element 0-1": np.abs(e01s),
        "T1 (us)": t1s,
        "EJ": [params[0] for params in paramss],
        "EC": [params[1] for params in paramss],
        "EL": [params[2] for params in paramss],
    }
)

# 添加標籤
data["Label"] = data.apply(
    lambda row: f"EJ={row['EJ']:.2f}, EC={row['EC']:.3f}, EL={row['EL']:.3f}", axis=1
)

# 繪製散點圖
fig = px.scatter(
    data,
    x="Matrix Element 0-1",
    y="T1 (us)",
    log_x=True,
    log_y=True,
    hover_name="Label",
    title=f"T1 vs Matrix Element 0-1 (Q_cap={Q_cap:.1e}, Q_ind={Q_ind:.1e}, x_qp={x_qp:.1e})",
    labels={"Matrix Element 0-1": "Matrix Element 0-1", "T1 (us)": "T1 (us)"},
)

max_id = np.argmax(t1s)

# Add annotation for the max_id point
max_point = data.iloc[max_id]
fig.add_annotation(
    x=np.log10(max_point["Matrix Element 0-1"]),  # Use log scale coordinates
    y=np.log10(max_point["T1 (us)"]),  # Use log scale coordinates
    text=max_point["Label"],
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,
    # Specify axes in terms of log scale
    xref="x",
    yref="y",
)

fig.update_layout(
    xaxis_title="Matrix Element 0-1",
    yaxis_title="T1 (us)",
    xaxis=dict(tickformat=".1e", type="log"),  # Ensure axis type is log
    yaxis=dict(tickformat=".1e", type="log"),  # Ensure axis type is log
    title_x=0.501,
    template="plotly_white",
    showlegend=True,
    width=900,
    height=750,
)

fig.show()

In [ ]:
fig.write_html(
    f"../../result/{qub_name}/web/t1vsM01_{max_id}.html", include_plotlyjs="cdn"
)
fig.write_image(f"../../result/{qub_name}/image/t1vsM01_{max_id}.png", format="png")

In [ ]:
EJ, EC, EL = paramss[max_id]
EJ, EC, EL

In [ ]:
params = (EJ, EC, EL)

flxs = np.linspace(0.0, 1.0, 300)

In [ ]:
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=40, truncated_dim=10)
spectrumData = fluxonium.get_matelements_vs_paramvals(
    operator="n_operator", param_name="flux", param_vals=flxs, evals_count=10
)

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(10) if j > i]

eval_eng = spectrumData.energy_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=eval_eng[:, j] - eval_eng[:, i],
            mode="lines",
            name=f"{i} - {j}",
            line=dict(width=2),
        )
    )
fig.add_hline(y=7.2, line_color="black", line_width=2, line_dash="dash")
fig.add_hline(y=5.9, line_color="black", line_width=2, line_dash="dash")
fig.update_yaxes(range=[0.0, 8.0])

fig.show()

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(3) if j > i]

matrixelements = spectrumData.matrixelem_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=np.abs(matrixelements[:, i, j]),
            mode="lines",
            name=f"{i}-{j}",
            line=dict(width=2),
        )
    )
fig.update_layout(
    title=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}",
    title_x=0.5,
    xaxis_title=r"$\phi_{ext}/\phi_0$",
    yaxis_title="Matrix elements",
)

fig.show()

In [ ]:
# Temp = 60e-3
# Q_cap = 1.0e5
# Q_ind = 1.0e7
# x_qp = 1.0e-8

scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=30, truncated_dim=5)
T1_eff_fig, T1_eff_ax = fluxonium.plot_t1_effective_vs_paramvals(
    param_name="flux",
    param_vals=flxs,
    xlim=([flxs.min(), flxs.max()]),
    common_noise_options=dict(i=1, j=0, T=Temp),
    noise_channels=[
        ("t1_capacitive", dict(Q_cap=Q_cap)),
        ("t1_inductive", dict(Q_ind=Q_ind)),
        ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
    ],
)
_ = T1_eff_ax.set_ylim(None, 1e6)